In [118]:
import pandas as pd
from tqdm import tqdm

In [119]:
name_input = "Finalver1.csv"
name_output = "Finalver2.csv"

In [120]:
df = pd.read_csv(name_input)
df.head()

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,quần áo,35.891 ₫,64% Off,3.0K Đã bán,(1022),Hồ Chí Minh,100.000 ₫,16.500 ₫,97%,100%,100%,20.0,7.0,27.0,36.0,932.0
1,quần áo,25.000 ₫,50% Off,907 Đã bán,(241),Hồ Chí Minh,71.998 ₫,16.500 ₫,91%,100%,50%,5.0,2.0,6.0,7.0,222.0
2,quần áo,79.000 ₫,43% Off,961 Đã bán,(260),Hà Nội,139.000 ₫,32.700 ₫,92%,99%,33%,11.0,8.0,8.0,16.0,218.0
3,quần áo,36.000 ₫,60% Off,102 Đã bán,(21),Nam Định,189.000 ₫,37.700 ₫,94%,100%,83%,1.0,0.0,1.0,1.0,18.0
4,quần áo,15.971 ₫,54% Off,219 Đã bán,(68),Hà Nội,59.000 ₫,32.700 ₫,95%,100%,100%,0.0,1.0,0.0,3.0,65.0


Các cột sẽ được LLMs biến đổi

In [121]:
selected_columns = ['Price_sale', 'Sale_off', 'Total_sold', 'Preview', 'Price_original', 'Ship_price', 'Sale_rating', 'Ship_on_time', 'Chat_response']

Dùng LLM: mistralai/Mixtral-8x22B-Instruct-v0.1

In [122]:
import os
import requests

s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_exa36aaw8u5pqdsppk8dqxi5ij"
url = f"{api_base}/chat/completions"

def convert_row(data):
    body = {
    "model": "mistralai/Mixtral-8x22B-Instruct-v0.1",
    "messages": [
        {
        "role": "user",
        "content": "bạn là trợ lý ảo về AI"
        },
        {
        "role": "assistant",
        "content": " Yes, I am an assistant that uses AI to help answer questions and provide information. I strive to always assist with care, respect, and truth, and to respond in a way that is most useful to you while also being secure. I avoid harmful, unethical, prejudiced, or negative content, and ensure that my replies promote fairness and positivity."
        },
        {
        "role": "user",
        "content": f"cho dữ liệu {data}. Hãy chuyển đổi thành số, loại bỏ ký tự đặc biệt, giá trị None thì ghi là None. Chỉ trả lời kết quả, không thêm bất kỳ câu khác"
        }
    ],
    "temperature": 0,
    "max_tokens": 256,
    "top_p": 1,
    "frequency_penalty": 0
    }

    with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
        res = resp.json()
        converted_content = res['choices'][0]['message']['content']
        return converted_content

In [123]:
# Hàm biến đổi theo dòng dựa vào cột đã chọn và cập nhật vào dataframe chính 
def transform_df(data):
    for i in tqdm(range(0, len(data))):
        row_data = data.loc[i, selected_columns]
        
        # Chuyển đổi các giá trị thành chuỗi và nối chúng với dấu phẩy
        input_ = ','.join(row_data.astype(str))
        result = convert_row(input_)
        # Chia chuỗi thành danh sách các phần tử
        values = result.split(", ")
        # Tạo dictionary với các key và giá trị tương ứng
        data_dict = {
            'Price_sale': values[0],
            'Sale_off': values[1],
            'Total_sold': values[2],
            'Preview': values[3],
            'Price_original': values[4],
            'Ship_price': values[5],
            'Sale_rating': values[6],
            'Ship_on_time': values[7],
            'Chat_response': values[8]
        }
        # Thay thế các giá trị trong dòng đó theo dictionary
        for key, value in data_dict.items():
            data.loc[i, key] = value
    

In [124]:
transform_df(df)

  0%|          | 3/7200 [00:16<10:56:10,  5.47s/it]


KeyboardInterrupt: 

In [ ]:
df.to_csv(name_output, index=False, encoding='utf-8-sig')